In [1]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de visualización
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# Importar módulos de la arquitectura
from config.database import db_config
from utils.data_loader import DataLoader
from utils.preproccesor import DataPreprocessor
from core.base_model import HoltWintersModel, ARIMAModel, EnsembleModel
from core.metrics import TimeSeriesMetrics, EducationalMetrics, AnomalyDetection
from utils.visualization import TimeSeriesVisualizer, EducationalVisualizer

print("✅ Módulos importados correctamente")

✅ Módulos importados correctamente


In [2]:
# Conectar a MongoDB
try:
    database = db_config.connect()
    print(f"✅ Conectado a la base de datos: {database.name}")
    
    # Obtener colecciones disponibles
    collections = database.list_collection_names()
    print(f"📚 Colecciones disponibles: {collections}")
    
except Exception as e:
    print(f"❌ Error conectando a la base de datos: {e}")
    # Crear datos de ejemplo para demostración
    print("🔄 Usando datos de ejemplo para demostración")

KeyboardInterrupt: 

In [ ]:
# Configurar fechas para el análisis
end_date = datetime.now()
start_date = end_date - timedelta(days=30)  # Últimos 30 días

print(f"📅 Período de análisis: {start_date.strftime('%Y-%m-%d')} a {end_date.strftime('%Y-%m-%d')}")

# Cargar datos de análisis de IA
try:
    ai_collection = db_config.get_collection('ai_analysis')
    ai_loader = DataLoader(ai_collection)
    ai_data = ai_loader.load_ai_analysis_data(start_date=start_date, end_date=end_date)
    print(f"🤖 Datos de análisis de IA cargados: {len(ai_data)} registros")
except Exception as e:
    print(f"⚠️ No se pudieron cargar datos de IA: {e}")
    ai_data = pd.DataFrame()

# Cargar datos de citas
try:
    appointments_collection = db_config.get_collection('appointments')
    appointments_loader = DataLoader(appointments_collection)
    appointments_data = appointments_loader.load_appointments_data(start_date=start_date, end_date=end_date)
    print(f"�� Datos de citas cargados: {len(appointments_data)} registros")
except Exception as e:
    print(f"⚠️ No se pudieron cargar datos de citas: {e}")
    appointments_data = pd.DataFrame()

# Cargar datos de tareas (desde el checklist de citas)
try:
    tasks_data = appointments_loader.load_tasks_data(start_date=start_date, end_date=end_date)
    print(f"📋 Datos de tareas cargados: {len(tasks_data)} registros")
except Exception as e:
    print(f"⚠️ No se pudieron cargar datos de tareas: {e}")
    tasks_data = pd.DataFrame()

In [ ]:
# Generar datos de ejemplo si no hay datos reales
if ai_data.empty:
    print("�� Generando datos de ejemplo para análisis de IA...")
    
    # Crear fechas de ejemplo
    dates = pd.date_range(start=start_date, end=end_date, freq='H')
    
    # Datos de análisis de IA
    ai_data = pd.DataFrame({
        'created_at': np.random.choice(dates, 100),
        'sender_id': [f'student_{i}' for i in np.random.randint(1, 21, 100)],
        'recipient_id': [f'tutor_{i}' for i in np.random.randint(1, 6, 100)],
        'message_content': [f'Mensaje de ejemplo {i}' for i in range(100)],
        'analysis.bullying': np.random.choice([True, False], 100, p=[0.1, 0.9]),
        'analysis.concern': np.random.choice([True, False], 100, p=[0.3, 0.7]),
        'analysis.academic_constructive': np.random.choice([True, False], 100, p=[0.8, 0.2]),
        'analysis.bullying_explanation': ['Sin bullying detectado'] * 100,
        'analysis.concern_explanation': ['Sin preocupaciones detectadas'] * 100,
        'analysis.academic_explanation': ['Conversación constructiva'] * 100
    })
    
    # Agregar longitud de mensaje
    ai_data['message_length'] = ai_data['message_content'].str.len()
    
    print(f"✅ Datos de ejemplo generados: {len(ai_data)} registros")

if appointments_data.empty:
    print("�� Generando datos de ejemplo para citas...")
    
    # Datos de citas
    appointment_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    appointments_data = pd.DataFrame({
        'fecha_cita': np.random.choice(appointment_dates, 50),
        'id_tutor': [f'tutor_{i}' for i in np.random.randint(1, 6, 50)],
        'id_alumno': [f'student_{i}' for i in np.random.randint(1, 21, 50)],
        'estado_cita': np.random.choice(['pendiente', 'confirmada', 'completada', 'cancelada', 'no_asistio'], 50, p=[0.2, 0.3, 0.3, 0.1, 0.1]),
        'reason': ['Consulta académica'] * 50,
        'checklist': [[{'description': f'Tarea {i}', 'completed': np.random.choice([True, False])} for i in range(np.random.randint(1, 4))] for _ in range(50)]
    })
    
    print(f"✅ Datos de citas generados: {len(appointments_data)} registros")

if tasks_data.empty:
    print("�� Generando datos de ejemplo para tareas...")
    
    # Extraer tareas del checklist de citas
    tasks_list = []
    for _, appointment in appointments_data.iterrows():
        for task in appointment['checklist']:
            tasks_list.append({
                'appointment_id': f'appointment_{len(tasks_list)}',
                'id_tutor': appointment['id_tutor'],
                'id_alumno': appointment['id_alumno'],
                'fecha_cita': appointment['fecha_cita'],
                'estado_cita': appointment['estado_cita'],
                'task_description': task['description'],
                'task_completed': task['completed']
            })
    
    tasks_data = pd.DataFrame(tasks_list)
    print(f"✅ Datos de tareas generados: {len(tasks_data)} registros")

In [ ]:
# Información general de los datasets
print("📊 RESUMEN DE DATOS")
print("=" * 50)

print(f"\n�� ANÁLISIS DE IA:")
print(f"   - Registros: {len(ai_data)}")
if not ai_data.empty:
    print(f"   - Columnas: {list(ai_data.columns)}")
    print(f"   - Período: {ai_data['created_at'].min()} a {ai_data['created_at'].max()}")
    print(f"   - Estudiantes únicos: {ai_data['sender_id'].nunique()}")
    print(f"   - Tutores únicos: {ai_data['recipient_id'].nunique()}")

print(f"\n📅 CITAS:")
print(f"   - Registros: {len(appointments_data)}")
if not appointments_data.empty:
    print(f"   - Columnas: {list(appointments_data.columns)}")
    print(f"   - Período: {appointments_data['fecha_cita'].min()} a {appointments_data['fecha_cita'].max()}")
    print(f"   - Estados: {appointments_data['estado_cita'].value_counts().to_dict()}")

print(f"\n�� TAREAS:")
print(f"   - Registros: {len(tasks_data)}")
if not tasks_data.empty:
    print(f"   - Columnas: {list(tasks_data.columns)}")
    print(f"   - Tasa de completado: {tasks_data['task_completed'].mean():.2%}")
    print(f"   - Tareas por cita: {len(tasks_data) / len(appointments_data):.1f}")

In [ ]:
# Inicializar preprocesador
preprocessor = DataPreprocessor()

# Limpiar datos de análisis de IA
if not ai_data.empty:
    print("�� Limpiando datos de análisis de IA...")
    ai_clean = preprocessor.clean_ai_analysis_data(ai_data)
    print(f"   - Registros originales: {len(ai_data)}")
    print(f"   - Registros limpios: {len(ai_clean)}")
    
    # Crear features adicionales
    ai_features = preprocessor.create_time_series_features(ai_clean, 'created_at')
    print(f"   - Features creados: {list(ai_features.columns)}")
else:
    ai_clean = pd.DataFrame()
    ai_features = pd.DataFrame()

# Limpiar datos de citas
if not appointments_data.empty:
    print("\n�� Limpiando datos de citas...")
    appointments_clean = preprocessor.clean_appointments_data(appointments_data)
    print(f"   - Registros originales: {len(appointments_data)}")
    print(f"   - Registros limpios: {len(appointments_clean)}")
else:
    appointments_clean = pd.DataFrame()

# Limpiar datos de tareas
if not tasks_data.empty:
    print("\n�� Limpiando datos de tareas...")
    tasks_clean = preprocessor.clean_tasks_data(tasks_data)
    print(f"   - Registros originales: {len(tasks_data)}")
    print(f"   - Registros limpios: {len(tasks_clean)}")
else:
    tasks_clean = pd.DataFrame()